## Deliverable 2. Create a Customer Travel Destinations Map.

In [48]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [49]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.51,77,100,8.93,light rain
1,1,Novo Aripuana,BR,-5.1206,-60.3797,74.25,95,98,1.28,overcast clouds
2,2,Aksarka,RU,66.5606,67.7975,14.04,88,96,4.50,overcast clouds
3,3,Pilar Do Sul,BR,-23.8131,-47.7164,58.93,94,35,6.73,scattered clouds
4,4,Bluff,NZ,-46.6000,168.3333,58.84,76,93,12.01,overcast clouds


In [50]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 68
What is the maximum temperature you would like for your trip? 95


In [51]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.51,77,100,8.93,light rain
1,1,Novo Aripuana,BR,-5.1206,-60.3797,74.25,95,98,1.28,overcast clouds
5,5,Cayenne,GF,4.9333,-52.3333,80.10,85,40,1.01,scattered clouds
7,7,Vaini,TO,-21.2000,-175.2000,77.16,100,90,2.30,moderate rain
8,8,Pahrump,US,36.2083,-115.9839,69.17,24,13,1.01,few clouds


In [52]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                300
City                   300
Country                298
Lat                    300
Lng                    300
Max Temp               300
Humidity               300
Cloudiness             300
Wind Speed             300
Current Description    300
dtype: int64

In [53]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.51,77,100,8.93,light rain
1,1,Novo Aripuana,BR,-5.1206,-60.3797,74.25,95,98,1.28,overcast clouds
5,5,Cayenne,GF,4.9333,-52.3333,80.10,85,40,1.01,scattered clouds
7,7,Vaini,TO,-21.2000,-175.2000,77.16,100,90,2.30,moderate rain
8,8,Pahrump,US,36.2083,-115.9839,69.17,24,13,1.01,few clouds


In [54]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.51,light rain,-23.1203,-134.9692,
1,Novo Aripuana,BR,74.25,overcast clouds,-5.1206,-60.3797,
5,Cayenne,GF,80.10,scattered clouds,4.9333,-52.3333,
7,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,
8,Pahrump,US,69.17,few clouds,36.2083,-115.9839,
9,Avarua,CK,73.45,scattered clouds,-21.2078,-159.7750,
10,Georgetown,MY,89.89,few clouds,5.4112,100.3354,
11,Atuona,PF,78.24,clear sky,-9.8000,-139.0333,
12,Dubbo,AU,82.53,scattered clouds,-32.2500,148.6167,
13,New Norfolk,AU,68.67,few clouds,-42.7826,147.0587,


In [55]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.51,light rain,-23.1203,-134.9692,People ThankYou
1,Novo Aripuana,BR,74.25,overcast clouds,-5.1206,-60.3797,POUSADA Flexy
5,Cayenne,GF,80.10,scattered clouds,4.9333,-52.3333,Hôtel Le Dronmi
7,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
8,Pahrump,US,69.17,few clouds,36.2083,-115.9839,
9,Avarua,CK,73.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
10,Georgetown,MY,89.89,few clouds,5.4112,100.3354,Cititel Penang
11,Atuona,PF,78.24,clear sky,-9.8000,-139.0333,Villa Enata
12,Dubbo,AU,82.53,scattered clouds,-32.2500,148.6167,Comfort Inn Blue Lagoon
13,New Norfolk,AU,68.67,few clouds,-42.7826,147.0587,The Shingles Riverside Cottages


In [56]:
# 7. Drop the rows where there is no Hotel Name.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        if hotels["results"][0]["name"] != "":
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")
   
# Replace all empty hotel names with NaNs so that dropna recognizes to drop it.
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna(axis=0, subset=["Hotel Name"])
clean_hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.51,light rain,-23.1203,-134.9692,People ThankYou
1,Novo Aripuana,BR,74.25,overcast clouds,-5.1206,-60.3797,POUSADA Flexy
5,Cayenne,GF,80.10,scattered clouds,4.9333,-52.3333,Hôtel Le Dronmi
7,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
8,Pahrump,US,69.17,few clouds,36.2083,-115.9839,Best Western Pahrump Oasis
9,Avarua,CK,73.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
10,Georgetown,MY,89.89,few clouds,5.4112,100.3354,Cititel Penang
11,Atuona,PF,78.24,clear sky,-9.8000,-139.0333,Villa Enata
12,Dubbo,AU,82.53,scattered clouds,-32.2500,148.6167,Comfort Inn Blue Lagoon
13,New Norfolk,AU,68.67,few clouds,-42.7826,147.0587,The Shingles Riverside Cottages


In [60]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [61]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [62]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))